<a href="https://colab.research.google.com/github/tmandingwa/tmandingwa/blob/main/FFC_Sensitivity_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 Sensitivity analysis using a Solver Table

That is, solving the optimization model multiple times as parameters change.

This set of codes guide you on how to perform sensitivity analysis to analyze how changes in model parameters affect your objective function and solution.

In [1]:
# Usual installation of packages and importing the packages we will use
!pip install -q pyomo
!apt-get install -y -qq glpk-utils
import pyomo.environ as pyo
import pandas as pd

Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 126718 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected package glpk-utils.
Preparing to unpack .../glpk-utils_5.0-1_amd64.deb ...
Unpacking glpk-utils (5.0-1) ...
Setting up libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4b

## 1.1 Your playground to a step-by-step approach to generate a solver table

In [3]:
# The usual model setup. For your reference.
model = pyo.ConcreteModel('My FFC optimisation model')
model.tables = pyo.Var(domain=pyo.NonNegativeReals, bounds = (100, None))
model.chairs = pyo.Var(domain=pyo.NonNegativeReals, bounds = (0, 450))
model.capentry = pyo.Constraint(expr = 3*model.tables + 4*model.chairs <= 2400)
model.painting = pyo.Constraint(expr = 2*model.tables + 1*model.chairs <= 1000)
model.profit = pyo.Objective(expr=7*model.tables + 5*model.chairs, sense=pyo.maximize)
solver = pyo.SolverFactory('glpk')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 4040.0, 'Upper bound': 4040.0, 'Number of objectives': 1, 'Number of constraints': 2, 'Number of variables': 2, 'Number of nonzeros': 4, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.0045931339263916016}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [4]:
# TO MODIFY: Steps
# 1. Wrap your codes in a function.
# 2. Update inputs. Labor, Machine.
# 3. Generate outputs. Labor Input, Machine Input, Labor Use, Machine Use, Optimal Cars, Optimal Trucks, Optimal Profit.
# def optimize_cars_and_trucks (labor = 500, machine = 800):
#   model = pyo.ConcreteModel('Cars and Trucks')
#   model.cars = pyo.Var(domain=pyo.NonNegativeReals)
#   model.trucks = pyo.Var(domain=pyo.NonNegativeReals)
#   model.labor = pyo.Constraint(expr=4 * model.cars + 6 * model.trucks <= labor)
#   model.machine = pyo.Constraint(expr=12 * model.cars + 8 * model.trucks <= machine)
#   model.profit = pyo.Objective(expr=5 * model.cars + 4 * model.trucks, sense=pyo.maximize)
#   solver = pyo.SolverFactory('glpk')
#   solver.solve(model)
#   return [labor, machine, model.labor(), model.machine(), model.cars(), model.trucks(), model.profit()]


#   import pyomo.environ as pyo

def optimize_tables_and_chairs(capentry_capacity=2400, painting_capacity=1000, chair_upper=450, table_lower=100):
    model = pyo.ConcreteModel()
    # Decision variables (with bounds)
    model.tables = pyo.Var(domain=pyo.NonNegativeReals, bounds=(table_lower, None))
    model.chairs = pyo.Var(domain=pyo.NonNegativeReals, bounds=(0, chair_upper))
    # Constraints
    model.capentry = pyo.Constraint(expr=3*model.tables + 4*model.chairs <= capentry_capacity)
    model.painting = pyo.Constraint(expr=2*model.tables + 1*model.chairs <= painting_capacity)
    # Objective (maximize profit)
    model.profit = pyo.Objective(expr=7*model.tables + 5*model.chairs, sense=pyo.maximize)
    # Solve
    solver = pyo.SolverFactory('glpk')
    solver.solve(model)
    # Return input scenario and solution
    return {
        "capentry_capacity": capentry_capacity,
        "painting_capacity": painting_capacity,
        "chairs_bound": chair_upper,
        "tables_bound": table_lower,
        "tables": pyo.value(model.tables),
        "chairs": pyo.value(model.chairs),
        "profit": pyo.value(model.profit),
        "capentry_used": 3*pyo.value(model.tables) + 4*pyo.value(model.chairs),
        "painting_used": 2*pyo.value(model.tables) + 1*pyo.value(model.chairs),
    }


In [7]:
# # Run the optimization over a range of capentry_capacity values for sensitivity analysis
results = [
    optimize_tables_and_chairs(capentry_capacity=cap)
    for cap in range(2400, 3000, 25)  # For example, from 1800 to 2500 in steps of 100
]

import pandas as pd
df = pd.DataFrame(results)
print(df)

# results = [
#     optimize_tables_and_chairs(painting_capacity=p)
#     for p in range(800, 1200, 50)
# ]

# df = pd.DataFrame(results)
# print(df)


    capentry_capacity  painting_capacity  chairs_bound  tables_bound  tables  \
0                2400               1000           450           100   320.0   
1                2425               1000           450           100   315.0   
2                2450               1000           450           100   310.0   
3                2475               1000           450           100   305.0   
4                2500               1000           450           100   300.0   
5                2525               1000           450           100   295.0   
6                2550               1000           450           100   290.0   
7                2575               1000           450           100   285.0   
8                2600               1000           450           100   280.0   
9                2625               1000           450           100   275.0   
10               2650               1000           450           100   275.0   
11               2675               1000

In [8]:
# Display your solutions
df = pd.DataFrame(results, columns=['capentry_capacity', 'painting_capacity', 'tables', 'chairs', 'profit'])
df

,capentry_capacity,painting_capacity,tables,chairs,profit
0,2400,1000,320.0,360.0,4040.0
1,2425,1000,315.0,370.0,4055.0
2,2450,1000,310.0,380.0,4070.0
3,2475,1000,305.0,390.0,4085.0
4,2500,1000,300.0,400.0,4100.0
5,2525,1000,295.0,410.0,4115.0
6,2550,1000,290.0,420.0,4130.0
7,2575,1000,285.0,430.0,4145.0
8,2600,1000,280.0,440.0,4160.0
9,2625,1000,275.0,450.0,4175.0
